# 목표

- 제주 태양광 발전소 위치 지도에 표시

# 초기 세팅

In [1]:
# 구글드라이브마운트하기

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 한글 나눔폰트 설치하기

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [3]:
# 한글 나눔폰트 설정하기

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

## 파일 가져오기 : 파일명에 속았다.. case

In [4]:
# 파일 가져오기

csvPath = '/content/drive/MyDrive/Colab Notebooks/Data Science/4_데이터분석예제/제주특별자치도_태양광발전소현황_20220630.csv'

In [15]:
import pandas as pd
df = pd.read_csv(csvPath, encoding='cp949')

In [16]:
df.head()

,행정시,읍면동,허가일자,상호,설비용량(KW),상태,사업개시일,데이터기준일자
0,제주시,구좌읍,1998-06-10,행원풍력발전단지(1차),3480.0,사업개시,2000-04-05,2022-06-30
1,제주시,회천동,2002-12-09,파낙스에너지(주),1000.0,사업개시,2002-12-18,2022-06-30
2,제주시,한경면,2003-04-14,한경풍력 1단계,6000.0,사업개시,2004-02-28,2022-06-30
3,제주시,한경면,2005-04-04,신창풍력발전단지,1700.0,사업개시,2006-03-03,2022-06-30
4,제주시,구좌읍,2005-08-30,제주월정풍력발전소,1500.0,사업개시,2006-07-20,2022-06-30


In [17]:
df.tail()

# 인허가 단계 발전소 : 아직 운영 X. 지어지지 않음

,행정시,읍면동,허가일자,상호,설비용량(KW),상태,사업개시일,데이터기준일자
2143,제주시,한경면,2022-06-08,민재 태양광발전소,99.6,인허가,NaN,2022-06-30
2144,제주시,한림읍,2022-06-08,효일4호 태양광발전소,99.0,인허가,NaN,2022-06-30
2145,제주시,한림읍,2022-06-08,효일3호 태양광발전소,99.0,인허가,NaN,2022-06-30
2146,제주시,한림읍,2022-06-08,효일5호 태양광발전소,99.0,인허가,NaN,2022-06-30
2147,서귀포시,성산읍,2022-06-21,한라2호소수력발전소,210.0,인허가,NaN,2022-06-30


In [18]:
df.info()

# 사업개시하지 않은 행은 데이터 분석 안됨 > 삭제
# 허가일자 datetime으로 변경

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2148 entries, 0 to 2147
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   행정시       2148 non-null   object 
 1   읍면동       2148 non-null   object 
 2   허가일자      2148 non-null   object 
 3   상호        2148 non-null   object 
 4   설비용량(KW)  2148 non-null   float64
 5   상태        2148 non-null   object 
 6   사업개시일     1571 non-null   object 
 7   데이터기준일자   2148 non-null   object 
dtypes: float64(1), object(7)
memory usage: 134.4+ KB


In [20]:
# df를 복제함. 결측치 있는 행 삭제

df1 = df.copy()
df1.dropna(inplace=True)

In [21]:
# 결측치 없는지 확인

df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1571 entries, 0 to 2091
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   행정시       1571 non-null   object 
 1   읍면동       1571 non-null   object 
 2   허가일자      1571 non-null   object 
 3   상호        1571 non-null   object 
 4   설비용량(KW)  1571 non-null   float64
 5   상태        1571 non-null   object 
 6   사업개시일     1571 non-null   object 
 7   데이터기준일자   1571 non-null   object 
dtypes: float64(1), object(7)
memory usage: 110.5+ KB


In [22]:
# 사업개시 안한 곳 아직있나? 확인
# 잘못된 데이터일수도

df1[df1['상태']!='사업개시']

,행정시,읍면동,허가일자,상호,설비용량(KW),상태,사업개시일,데이터기준일자
6,제주시,한경면,2006-08-11,탐라해상풍력발전소,30000.0,공사진행,2017-09-16,2022-06-30
29,서귀포시,성산읍,2008-02-28,성산풍력 1단계,12000.0,공사진행,2009-03-30,2022-06-30
583,제주시,한경면,2016-11-16,성용유화발전,750.0,인허가,2016-11-16,2022-06-30


In [23]:
# 위의 장소들은 이미 사업개시한 곳들임
# 상태값 사업개시로 변경해주기

editList = df1[df1['상태']!='사업개시'].index

for index in editList:
  df1.loc[index, '상태'] = '사업개시'

In [24]:
# 다시 확인

df1[df1['상태']!='사업개시']

,행정시,읍면동,허가일자,상호,설비용량(KW),상태,사업개시일,데이터기준일자


In [25]:
df1.head()

,행정시,읍면동,허가일자,상호,설비용량(KW),상태,사업개시일,데이터기준일자
0,제주시,구좌읍,1998-06-10,행원풍력발전단지(1차),3480.0,사업개시,2000-04-05,2022-06-30
1,제주시,회천동,2002-12-09,파낙스에너지(주),1000.0,사업개시,2002-12-18,2022-06-30
2,제주시,한경면,2003-04-14,한경풍력 1단계,6000.0,사업개시,2004-02-28,2022-06-30
3,제주시,한경면,2005-04-04,신창풍력발전단지,1700.0,사업개시,2006-03-03,2022-06-30
4,제주시,구좌읍,2005-08-30,제주월정풍력발전소,1500.0,사업개시,2006-07-20,2022-06-30


## 파일 가져오기 : 제대로된 파일 다시 가져옴

In [26]:
csvPath = '/content/drive/MyDrive/Colab Notebooks/Data Science/4_데이터분석예제/제주특별자치도_신재생에너지발전시설현황_20221116.csv'

In [27]:
import pandas as pd
df = pd.read_csv(csvPath, encoding='cp949')

In [28]:
df.head()

,허가일자,상호,설비용량(KW),설치장소,원동력종류,사업개시일,데이터기준일자
0,2019-05-22,북촌서모풍력발전소,3000.0,제주특별자치도 제주시 조천읍,풍력,2020-07-31,2022-11-16
1,2016-12-12,동복리마을풍력발전소,2000.0,제주특별자치도 제주시 구좌읍,풍력,2017-12-01,2022-11-16
2,2016-10-17,수망풍력발전소,25200.0,제주특별자치도 서귀포시 남원읍,풍력,2020-06-29,2022-11-16
3,2014-06-13,동복북촌풍력발전단지,30000.0,제주특별자치도 제주시 구좌읍,풍력,2015-08-01,2022-11-16
4,2014-05-02,상명풍력발전소,21000.0,제주특별자치도 제주시 한림읍,풍력,2016-08-22,2022-11-16


In [29]:
df.tail()

,허가일자,상호,설비용량(KW),설치장소,원동력종류,사업개시일,데이터기준일자
1577,2013-03-12,주식회사 이웰,2950.0,제주특별자치도 서귀포시 토평동,바이오가스,2015-07-21,2022-11-16
1578,2012-11-27,농업회사법인 주식회사 한라산 바이오,500.0,제주특별자치도 서귀포시 안덕면,바이오가스,2015-05-01,2022-11-16
1579,2010-06-21,(주)농업회사법인 제주축산바이오가스발전소,99.0,제주특별자치도 제주시 한림읍,바이오가스,2010-08-24,2022-11-16
1580,2002-12-09,파낙스에너지(주),1000.0,제주특별자치도 제주시 회천동,바이오가스,2002-12-18,2022-11-16
1581,2015-05-27,서광산업 신재생에너지발전소,2400.0,제주특별자치도 제주시 애월읍,기타,2015-12-17,2022-11-16


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   허가일자      1582 non-null   object 
 1   상호        1582 non-null   object 
 2   설비용량(KW)  1582 non-null   float64
 3   설치장소      1582 non-null   object 
 4   원동력종류     1582 non-null   object 
 5   사업개시일     1582 non-null   object 
 6   데이터기준일자   1582 non-null   object 
dtypes: float64(1), object(6)
memory usage: 86.6+ KB


In [31]:
df['원동력종류'].unique()

array(['풍력', '태양광', '수력', '바이오가스', '기타'], dtype=object)

In [36]:
df2 = df[df['원동력종류']=='태양광']

df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1547 entries, 24 to 1570
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   허가일자      1547 non-null   object 
 1   상호        1547 non-null   object 
 2   설비용량(KW)  1547 non-null   float64
 3   설치장소      1547 non-null   object 
 4   원동력종류     1547 non-null   object 
 5   사업개시일     1547 non-null   object 
 6   데이터기준일자   1547 non-null   object 
dtypes: float64(1), object(6)
memory usage: 96.7+ KB


In [37]:
df2.head()

,허가일자,상호,설비용량(KW),설치장소,원동력종류,사업개시일,데이터기준일자
24,2021-09-15,흥산태양광발전소,999.63,제주특별자치도 제주시 구좌읍,태양광,2022-04-25,2022-11-16
25,2021-08-03,태양7호태양광발전소,198.88,제주특별자치도 서귀포시 안덕면,태양광,2021-12-10,2022-11-16
26,2021-04-21,제주덕암 태양광발전소,99.18,제주특별자치도 서귀포시 표선면,태양광,2022-01-03,2022-11-16
27,2021-04-19,청춘 태양광발전소,662.48,제주특별자치도 서귀포시 안덕면,태양광,2022-01-03,2022-11-16
28,2021-04-19,에스원태양광발전소,458.64,제주특별자치도 서귀포시 안덕면,태양광,2022-01-03,2022-11-16


In [39]:
df2.tail()

,허가일자,상호,설비용량(KW),설치장소,원동력종류,사업개시일,데이터기준일자
1566,2007-02-21,제주에너지 주식회사,200.00,제주특별자치도 서귀포시 성산읍,태양광,2008-09-29,2022-11-16
1567,2007-02-21,제주에너지 주식회사,200.00,제주특별자치도 제주시 애월읍,태양광,2008-04-18,2022-11-16
1568,2007-01-05,민솔라에너지,90.72,제주특별자치도 제주시 조천읍,태양광,2007-04-04,2022-11-16
1569,2006-08-24,오복산전 3호,29.52,제주특별자치도 제주시 한경면,태양광,2006-09-25,2022-11-16
1570,2006-03-03,오복산전,59.87,제주특별자치도 제주시 한경면,태양광,2006-07-11,2022-11-16


## 데이터가 얼마나 신뢰성있는가? 확인

- 데이터 신뢰성 검사
- 다른 곳에 같은 데이터를찾아보고 동일한지 체크

# 설비용량 계산

In [42]:
# 태양광 - 설비용량 계산

df2.columns

Index(['허가일자', '상호', '설비용량(KW)', '설치장소', '원동력종류', '사업개시일', '데이터기준일자'], dtype='object')

In [45]:
# 2022.11.16 기준

df2['설비용량(KW)'].sum()

509086.72

# 설치장소 파악

In [46]:
# 읍면동 단위까지 밖에 안나옴
# 태양광발전소가 설치되어있는 읍면동 찾기(중복제거) + 몇개인지?

print(df2['설치장소'].unique())
print(df2['설치장소'].nunique())


['제주특별자치도 제주시 구좌읍' '제주특별자치도 서귀포시 안덕면' '제주특별자치도 서귀포시 표선면'
 '제주특별자치도 서귀포시 성산읍' '제주특별자치도 서귀포시 법환동' '제주특별자치도 서귀포시 대정읍'
 '제주특별자치도 제주시 한림읍' '제주특별자치도 서귀포시 남원읍' '제주특별자치도 서귀포시 상예동'
 '제주특별자치도 서귀포시 회수동' '제주특별자치도 제주시 조천읍' '제주특별자치도 제주시 도남동' '제주특별자치도 제주시 애월읍'
 '제주특별자치도 서귀포시 토평동' '제주특별자치도 서귀포시 강정동' '제주특별자치도 서귀포시 중문동'
 '제주특별자치도 제주시 봉개동' '제주특별자치도 서귀포시 월평동' '제주특별자치도 제주시 용담이동'
 '제주특별자치도 제주시 일도이동' '제주특별자치도 제주시 회천동' '제주특별자치도 제주시 해안동' '제주특별자치도 서귀포시 호근동'
 '제주특별자치도 제주시 오라삼동' '제주특별자치도 제주시 한경면' '제주특별자치도 제주시 아라이동'
 '제주특별자치도 제주시 오라이동' '제주특별자치도 제주시 용강동' '제주특별자치도 제주시 외도일동'
 '제주특별자치도 제주시 삼양일동' '제주특별자치도 서귀포시 도순동' '제주특별자치도 서귀포시 상효동'
 '제주특별자치도 서귀포시 서홍동' '제주특별자치도 제주시 이도일동' '제주특별자치도 제주시 오등동'
 '제주특별자치도 서귀포시 서호동' '제주특별자치도 제주시 영평동' '제주특별자치도 서귀포시 동홍동'
 '제주특별자치도 서귀포시 하원동' '제주특별자치도 제주시 이호이동' '제주특별자치도 제주시 화북이동'
 '제주특별자치도 서귀포시 색달동' '제주특별자치도 서귀포시 서귀동' '제주특별자치도 제주시 도련이동'
 '제주특별자치도 제주시 오라일동' '제주특별자치도 서귀포시 하효동' '제주특별자치도 제주시 도평동'
 '제주특별자치도 서귀포시 하예동' '제주특별자치도 제주시 노형동' '제주특별자치도 제주시 삼도이동'
 '제주특별자치도 제주시 도련일동' '제주특별자치도 제주시 아라일동']
52


# 설비용량 파악

In [49]:
# 태양광 발전소가 1년간 새로 사업 시작한 갯수
# 발전용량은 얼마나 신축되었는가

df_year = df2[(df2['사업개시일'] >= '2021-01-01')&(df2['사업개시일'] <= '2021-12-31')]
df_year['설비용량(KW)'].sum()

108352.51999999999

# 주소 변환 (주소 > 위도경도)

In [50]:
# geopy 패키지 있는지 확인
# 주소값을 위경도로 변환

!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohappyeyeballs                 2.3.4
aiohttp                          3.10.1
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.13
albumentations                   1.4.13
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.2
astropy-iers-data                0.2024.8.5.0.32.23
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            24.2.0
audioread               

In [55]:
# 참고용 geopy 코드


# 가입 없이 주소->좌표 변환
from geopy.geocoders import Nominatim

# 주소나 위도, 경도가 중복되지 않은 고유한 정보를
# 리스트에 각각 담아주기 위해 빈 리스트 생성
unique_addr=[]
unique_lat=[]
unique_lng=[]
except_addr=[]

# 주소를 위도와 경도로 변환해 주는 함수
def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    # 주소값을 제대로 인식하지 못할때
    # 위도와 경도값을 None값을 리턴한다.
    if geo==None:
        print('주소가 어디인지 인식할 수 없습니다.:{}'.address)
        except_addr.append(address)
        return 0, 0
    else:
        return geo.latitude, geo.longitude

In [63]:
# geopy test

geo = geocoding("제주시 삼도 2동")
print(geo)

(33.5041775, 126.5202364)


In [57]:
# 데이터프레임 ['설치장소']['경도']['위도']

mapAddr_df = df2['설치장소'].unique()
print(mapAddr_df)

['제주특별자치도 제주시 구좌읍' '제주특별자치도 서귀포시 안덕면' '제주특별자치도 서귀포시 표선면'
 '제주특별자치도 서귀포시 성산읍' '제주특별자치도 서귀포시 법환동' '제주특별자치도 서귀포시 대정읍'
 '제주특별자치도 제주시 한림읍' '제주특별자치도 서귀포시 남원읍' '제주특별자치도 서귀포시 상예동'
 '제주특별자치도 서귀포시 회수동' '제주특별자치도 제주시 조천읍' '제주특별자치도 제주시 도남동' '제주특별자치도 제주시 애월읍'
 '제주특별자치도 서귀포시 토평동' '제주특별자치도 서귀포시 강정동' '제주특별자치도 서귀포시 중문동'
 '제주특별자치도 제주시 봉개동' '제주특별자치도 서귀포시 월평동' '제주특별자치도 제주시 용담이동'
 '제주특별자치도 제주시 일도이동' '제주특별자치도 제주시 회천동' '제주특별자치도 제주시 해안동' '제주특별자치도 서귀포시 호근동'
 '제주특별자치도 제주시 오라삼동' '제주특별자치도 제주시 한경면' '제주특별자치도 제주시 아라이동'
 '제주특별자치도 제주시 오라이동' '제주특별자치도 제주시 용강동' '제주특별자치도 제주시 외도일동'
 '제주특별자치도 제주시 삼양일동' '제주특별자치도 서귀포시 도순동' '제주특별자치도 서귀포시 상효동'
 '제주특별자치도 서귀포시 서홍동' '제주특별자치도 제주시 이도일동' '제주특별자치도 제주시 오등동'
 '제주특별자치도 서귀포시 서호동' '제주특별자치도 제주시 영평동' '제주특별자치도 서귀포시 동홍동'
 '제주특별자치도 서귀포시 하원동' '제주특별자치도 제주시 이호이동' '제주특별자치도 제주시 화북이동'
 '제주특별자치도 서귀포시 색달동' '제주특별자치도 서귀포시 서귀동' '제주특별자치도 제주시 도련이동'
 '제주특별자치도 제주시 오라일동' '제주특별자치도 서귀포시 하효동' '제주특별자치도 제주시 도평동'
 '제주특별자치도 서귀포시 하예동' '제주특별자치도 제주시 노형동' '제주특별자치도 제주시 삼도이동'
 '제주특별자치도 제주시 도련일동' '제주특별자치도 제주시 아라일동']


In [64]:
unique_lat=[]
unique_lng=[]

for addr in mapAddr_df :

    if addr=="제주특별자치도 제주시 삼도이동" :
        addr = "제주시 삼도 2동"
    geo = geocoding(addr)
    unique_addr.append(addr)
    print(geo)
    unique_lat.append(geo[0])
    unique_lng.append(geo[1])

(33.51662, 126.81055)
(33.27688, 126.34925)
(33.3504543, 126.794049)
(33.41805, 126.88147)
(33.24113, 126.51285)
(33.2218157, 126.2529945)
(33.39344, 126.26684)
(33.3076683, 126.6907475)
(33.26177, 126.38678)
(33.26549, 126.44363)
(33.50888, 126.66305)
(33.4905148, 126.5239964)
(33.45079, 126.3752)
(33.2705286, 126.5841149)
(33.24695, 126.48927)
(33.262002, 126.4288617)
(33.484188, 126.5993497)
(33.24501, 126.46294)
(33.50742, 126.50615)
(33.50864, 126.53795)
(33.4956, 126.61479)
(33.44899, 126.46036)
(33.25632, 126.53351)
(33.49487, 126.50508)
(33.3305, 126.21066)
(33.48148, 126.55055)
(33.47081, 126.50991)
(33.47017, 126.58934)
(33.4855847, 126.4284514)
(33.52345, 126.599)
(33.2569662, 126.4731036)
(33.2896254, 126.5945088)
(33.25965, 126.55175)
(33.5059, 126.5278)
(33.46148, 126.53087)
(33.25759, 126.51916)
(33.4812, 126.57106)
(33.26184, 126.56804)
(33.2626587, 126.4600239)
(33.498551, 126.4565213)
(33.50469, 126.5648)
(33.2640931, 126.4120002)
(33.24679, 126.56396)
(33.51279, 126.

In [66]:
map_df = pd.DataFrame({'설치장소':df2['설치장소'].unique(),
                       '위도':unique_lat,
                       '경도':unique_lng})

In [67]:
len(df2['설치장소'].unique()), len(unique_lat), len(unique_lng)

(52, 52, 52)

In [68]:
map_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   설치장소    52 non-null     object 
 1   위도      52 non-null     float64
 2   경도      52 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.3+ KB


# 지도 표시

In [69]:
# map_df['설치장소']를 index로 설정
map_df.set_index('설치장소', inplace=True)

In [70]:
map_df.head()

,위도,경도
설치장소,,
제주특별자치도 제주시 구좌읍,33.516620,126.810550
제주특별자치도 서귀포시 안덕면,33.276880,126.349250
제주특별자치도 서귀포시 표선면,33.350454,126.794049
제주특별자치도 서귀포시 성산읍,33.418050,126.881470
제주특별자치도 서귀포시 법환동,33.241130,126.512850


In [71]:

# 지도를 그릴때 사용하는 패키지는 folium

import pandas as pd
import folium

# 지도가 그려질 중심 좌표(대략 제주도의 중심 좌표), 지도를 그릴때 얼마나 줌을 적용할지
jeju_map = folium.Map(location=[33.38, 126.55], zoom_start=11)

for i in range(len(map_df)):
    pos_list = [map_df.iloc[i][0], map_df.iloc[i][1]]
    folium.Marker(pos_list, popup=map_df.index[i]).add_to(jeju_map)

jeju_map.save('jeju.html')

<ipython-input-71-07a092f42839>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pos_list = [map_df.iloc[i][0], map_df.iloc[i][1]]
